In [1]:
import pandas as pd

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)




In [3]:
with open('input.txt','r',encoding='utf-8') as f:
    lines=f.read().split('\n')

In [4]:
print(lines[155])

It's me!	Ich bin's.


In [5]:
len(lines)

169813

In [21]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [22]:
num_samples = 10000
num_samples

10000

In [23]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [24]:
input_texts[155],target_texts[155]

("It's me!", "\tIch bin's.\n")

In [25]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 90
Max sequence length for inputs: 16
Max sequence length for outputs: 53


In [26]:
input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])

In [27]:
import numpy as np

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

In [28]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
          # decoder_target_data will be ahead by one timestep
          # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [29]:
import keras,tensorflow
from keras.models import Model
from keras.layers import Input,LSTM,Dense


In [30]:
batch_size = 64  # batch size for training
epochs = 100  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space

In [31]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [32]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [33]:
model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)

In [34]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 70)     0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None, 90)     0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 334848      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  355328      input_4[0][0]                    
                                                                 lstm_3[0][1]               

In [20]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs)

Epoch 1/100
10000/10000 [==============================] - 8s 806us/step - loss: 1.0534
Epoch 2/100
10000/10000 [==============================] - 7s 719us/step - loss: 0.7820
Epoch 3/100
10000/10000 [==============================] - 8s 841us/step - loss: 0.6554
Epoch 4/100
10000/10000 [==============================] - 8s 800us/step - loss: 0.5934
Epoch 5/100
10000/10000 [==============================] - 7s 742us/step - loss: 0.5523
Epoch 6/100
10000/10000 [==============================] - 8s 756us/step - loss: 0.5193
Epoch 7/100
10000/10000 [==============================] - 8s 760us/step - loss: 0.4934
Epoch 8/100
10000/10000 [==============================] - 8s 761us/step - loss: 0.4712
Epoch 9/100
10000/10000 [==============================] - 8s 750us/step - loss: 0.4521
Epoch 10/100
10000/10000 [==============================] - 7s 744us/step - loss: 0.4349
Epoch 11/100
10000/10000 [==============================] - 7s 740us/step - loss: 0.4198
Epoch 12/100
10000/10000 [====

10000/10000 [==============================] - 8s 759us/step - loss: 0.0724
Epoch 93/100
10000/10000 [==============================] - 8s 782us/step - loss: 0.0714
Epoch 94/100
10000/10000 [==============================] - 9s 870us/step - loss: 0.0700 
Epoch 95/100
10000/10000 [==============================] - 8s 807us/step - loss: 0.0691
Epoch 96/100
10000/10000 [==============================] - 8s 752us/step - loss: 0.0681
Epoch 97/100
10000/10000 [==============================] - 8s 751us/step - loss: 0.0667
Epoch 98/100
10000/10000 [==============================] - 7s 744us/step - loss: 0.0661
Epoch 99/100
10000/10000 [==============================] - 7s 745us/step - loss: 0.0652
Epoch 100/100
10000/10000 [==============================] - 7s 742us/step - loss: 0.0641


In [35]:
 model.fit([encoder_input_data, decoder_input_data],
           decoder_target_data, batch_size=batch_size,
           epochs=epochs, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 7s 864us/step - loss: 1.0393 - val_loss: 1.0502
Epoch 2/100
8000/8000 [==============================] - 6s 809us/step - loss: 0.8035 - val_loss: 0.8464
Epoch 3/100
8000/8000 [==============================] - 7s 852us/step - loss: 0.6746 - val_loss: 0.7595
Epoch 4/100
8000/8000 [==============================] - 6s 792us/step - loss: 0.6098 - val_loss: 0.7129
Epoch 5/100
8000/8000 [==============================] - 6s 776us/step - loss: 0.5696 - val_loss: 0.6802
Epoch 6/100
8000/8000 [==============================] - 6s 777us/step - loss: 0.5364 - val_loss: 0.6494
Epoch 7/100
8000/8000 [==============================] - 6s 788us/step - loss: 0.5083 - val_loss: 0.6275
Epoch 8/100
8000/8000 [==============================] - 6s 790us/step - loss: 0.4841 - val_loss: 0.6158
Epoch 9/100
8000/8000 [==============================] - 6s 800us/step - loss: 0.4630 - val_loss: 0.5911
Epoch 1

8000/8000 [==============================] - 7s 832us/step - loss: 0.0828 - val_loss: 0.7761
Epoch 79/100
8000/8000 [==============================] - 7s 839us/step - loss: 0.0811 - val_loss: 0.7822
Epoch 80/100
8000/8000 [==============================] - 7s 853us/step - loss: 0.0803 - val_loss: 0.7825
Epoch 81/100
8000/8000 [==============================] - 7s 837us/step - loss: 0.0785 - val_loss: 0.7887
Epoch 82/100
8000/8000 [==============================] - 6s 806us/step - loss: 0.0772 - val_loss: 0.8007
Epoch 83/100
8000/8000 [==============================] - 7s 834us/step - loss: 0.0764 - val_loss: 0.8003
Epoch 84/100
8000/8000 [==============================] - 7s 861us/step - loss: 0.0747 - val_loss: 0.8004
Epoch 85/100
8000/8000 [==============================] - 6s 790us/step - loss: 0.0734 - val_loss: 0.8121
Epoch 86/100
8000/8000 [==============================] - 6s 788us/step - loss: 0.0724 - val_loss: 0.8147
Epoch 87/100
8000/8000 [==============================] - 6